# pyobsplot

**Note :** this notebook is designed to be used on [Google Colab](https://colab.research.google.com/github/juba/pyobsplot/blob/main/examples/introduction.ipynb).

[pyobsplot](https://github.com/juba/pyobsplot) is a Python package which allows to use Observable Plot in Jupyter notebooks with a syntax as close as possible to the JavaScript one. For more information, see the [documentation website](https://juba.github.io/pyobsplot).

## Getting started

First, we install the package:

In [ ]:
# Only needed in Colab, cleanup environment
! pip uninstall -y pandas-gbq
# Install pyobsplot
! pip install pyobsplot

For the simplest case we only need to import the `Plot` class:

In [1]:
from pyobsplot import Plot

We can then generate our first plots by passing a plot specification to the `Plot.plot()` method:

In [ ]:
import polars as pl
penguins = pl.read_csv("https://github.com/juba/pyobsplot/raw/main/doc/data/penguins.csv")

Plot.plot({
    "grid": True,
    "marks": [
        Plot.dot(penguins, {
            "x": "culmen_length_mm", "y": "culmen_depth_mm", "fill": "island", "tip": True
        })
    ]
})

In [ ]:
Plot.plot({
  "grid": True,
  "marginRight": 80,
  "color": {"legend": True},
  "marks": [
    Plot.rectY(penguins, Plot.binX({"y": "count"}, {"x": "body_mass_g", "fill": "island", "fy": "island"})),
    Plot.ruleY([0])
  ]
})

## Generator object

Using `Plot.plot()` allows to create plots with the default settings. To go a bit further we can create a *plot generator object* by importing the `Obsplot` class and calling it:

In [ ]:
from pyobsplot import Obsplot, Plot

op = Obsplot()

To produce plots with this generator object, we pass it a specification like we did with `Plot.plot()`:

In [ ]:
op({
    "color": {"legend": True},
    "marginLeft": 80,
    "marginRight": 80,
    "x": {"inset": 20},
    "grid": True,
    "marks": [
        Plot.boxX(penguins, {
            "x": "body_mass_g", "fill": "island", "y": "island", "fy": "species"
        })
    ]
})

A generator object allows, for example, to directly call a mark function:

In [ ]:
op(Plot.auto(penguins, {"x": "body_mass_g"}))

## jsdom renderer

By default, plots are created as Jupyter widgets, using the `widget` renderer. `pyobsplot` provides another renderer, called `jsdom`, which generates plots directly as SVG or HTML.

To use this renderer, we need to have a working installation of a recent node.js version:

In [ ]:
! apt-get install -y ca-certificates curl gnupg
! mkdir -p /etc/apt/keyrings
! curl -fsSL https://deb.nodesource.com/gpgkey/nodesource-repo.gpg.key | sudo gpg --dearmor -o /etc/apt/keyrings/nodesource.gpg
! echo "deb [signed-by=/etc/apt/keyrings/nodesource.gpg] https://deb.nodesource.com/node_18.x nodistro main" | sudo tee /etc/apt/sources.list.d/nodesource.list
! apt-get update
! apt-get install -y nodejs
! npm install -g npm

And we need to install the `pyobsplot` npm package:

In [ ]:
! npm install pyobsplot

Then we can create a new plot generator object, this time using the `jsdom` renderer:

In [ ]:
op_jsdom = Obsplot(renderer="jsdom")

The plots generated with this object will be created directly as SVG or HTML.

In [ ]:
ca55 = pl.read_csv("https://github.com/juba/pyobsplot/raw/main/doc/data/ca55-south.csv")

op_jsdom({
    "x": {"axis": None},
    "y": {"axis": None},
    "inset": 10,
    "marginBottom": 2,
    "height": 500,
    "color": {"type": "diverging"},
    "marks": [
        Plot.raster(
            ca55,
            {
                "x": "LONGITUDE",
                "y": "LATITUDE",
                "fill": "MAG_IGRF90",
                "interpolate": "random-walk",
            },
        ),
        Plot.frame(),
    ],
})


Sometimes a plot specification includes some JavaScript code. To add those to a plot specification we have to pass it as a string and wrap it into the `js()` method:

In [ ]:
from pyobsplot import js
from datetime import datetime

ipos = pl.read_csv("https://github.com/juba/pyobsplot/raw/main/doc/data/ipos.csv", try_parse_dates=True).filter(
    pl.col("date") > datetime(1991, 1, 1)
)


op_jsdom({
    "insetRight": 10,
    "height": 600,
    "width": 600,
    "marks": [
        Plot.dot(
            ipos, Plot.dodgeY({"x": "date", "r": "rMVOP", "fill": "currentColor"})
        ),
        Plot.text(
            ipos,
            Plot.dodgeY(
                {
                    "filter": js("(d) => d.rMVOP > 5e3"),
                    "x": "date",
                    "r": "rMVOP",
                    "text": js("d => (d.rMVOP / 1e3).toFixed()"),
                    "fill": "white",
                    "fontWeight": "bold",
                }
            ),
        ),
    ],
})